In [1]:
import re
import requests
import time
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

In [3]:
categories = [
    "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/",
    "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/",
    "https://www.amazon.com/Best-Sellers/zgbs/fashion/"
]

In [4]:
# categories

In [5]:
first_url = categories[0]

In [6]:
driver.get(first_url)

In [7]:
body_el = driver.find_element_by_css_selector("body")
html_str = body_el.get_attribute("innerHTML")

In [8]:
html_obj = HTML(html=html_str)

In [9]:
page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
# new_links = [x for x in new_links if "product-reviews/" not in x]

In [10]:
# page_links

In [11]:
def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    time.sleep(0.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [12]:
# https://www.amazon.com/LEGO-Classic-Medium-Creative-Brick/dp/B00NHQFA1I/
# https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/

# <base-url>/<slug>/dp/<product_id>/

In [13]:
# my_regex_pattern = r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
# my_url = 'https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/'

In [14]:
# regex = re.compile(my_regex_pattern)

In [15]:
# my_match = regex.match(my_url)
# print(my_match)
# my_match['product_id']

In [16]:
# my_match['slug']

In [17]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [18]:

# page_links = [x for x in page_links if extract_product_id_from_url(x) != None]
def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id})
    return final_page_links

cleaned_links = clean_page_links(page_links)

In [19]:
len(page_links) # == len(cleaned_links)

205

In [20]:
len(cleaned_links)

51

In [21]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {
            "url": link,
            "product_id": product_id,
            "title": title,
            "price": price
        }
        data_extracted.append(product_data)
    return data_extracted

In [22]:
extracted_data = perform_scrape(cleaned_items=cleaned_links)

https://www.amazon.com/Crayola-Colored-Pre-sharpened-Coloring-Stocking/dp/B018HB2QFU/ref=zg_bs_toys-and-games_10/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Crayola Colored Pencils, Adult Coloring, Fun At Home Activities, 50 Count $19.38
https://www.amazon.com/Little-Tikes-Spiralin-Waterpark-Table/dp/B004INDQWY/ref=zg_bs_toys-and-games_37/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Little Tikes Spiralin’ Seas Waterpark Water Table $44.99
https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/ref=zg_bs_toys-and-games_21/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Crayola Washable Watercolors, Paint Set For Kids, 8Count $1.99
https://www.amazon.com/Munchkin-17040-Fishin-Bath-Toy/dp/B01N52DUNK/ref=zg_bs_toys-and-games_48/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Munchkin Fishin' Bath Toy $7.73
https://www.amazon.com/Hasbro-B0965-Monopoly-Deal-Card/dp/B00NQQTZCO/ref=zg_bs_t

https://www.amazon.com/Crayola-Outdoor-Glitter-Sidewalk-Summer/dp/B00PY47F8M/ref=zg_bs_toys-and-games_16/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Crayola Outdoor Chalk, Glitter Sidewalk Chalk, Summer Toys, 5 Count $8.73
https://www.amazon.com/Creative-Roots-Stepping-Horizon-Assorted/dp/B07HSJ5R7V/ref=zg_bs_toys-and-games_50/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD CREATIVE ROOTS Paint Your Own Flower Stepping Stone by Horizon Group USA Toy, Assorted $7.99
https://www.amazon.com/Kinetic-Sand-Beach-Kingdom-Playset/dp/B079P87RVB/ref=zg_bs_toys-and-games_41/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Kinetic Sand Beach Sand Kingdom Playset with 3lbs of Beach Sand, for Ages 3 and Up $19.97
https://www.amazon.com/Bunch-Balloons-Pack-Amazon-Exclusive/dp/B07GW2QQWN/ref=zg_bs_toys-and-games_28/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD Bunch O Balloons - 350 Rapid-Fill Water Balloons (10 P

In [23]:
print(extracted_data)

[{'url': 'https://www.amazon.com/Crayola-Colored-Pre-sharpened-Coloring-Stocking/dp/B018HB2QFU/ref=zg_bs_toys-and-games_10/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD', 'product_id': 'B018HB2QFU', 'title': 'Crayola Colored Pencils, Adult Coloring, Fun At Home Activities, 50 Count', 'price': '$19.38'}, {'url': 'https://www.amazon.com/Little-Tikes-Spiralin-Waterpark-Table/dp/B004INDQWY/ref=zg_bs_toys-and-games_37/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD', 'product_id': 'B004INDQWY', 'title': 'Little Tikes Spiralin’ Seas Waterpark Water Table', 'price': '$44.99'}, {'url': 'https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/ref=zg_bs_toys-and-games_21/138-0291413-2906220?_encoding=UTF8&psc=1&refRID=PZN4KF19MNZDRXDYK2WD', 'product_id': 'B000HHKAE2', 'title': 'Crayola Washable Watercolors, Paint Set For Kids, 8Count', 'price': '$1.99'}, {'url': 'https://www.amazon.com/Munchkin-17040-Fishin-Bath-Toy/dp/B01N52DUNK/ref=zg_